<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03

In [7]:
# TensorFlow와 NumPy 불러오기
import tensorflow as tf
import numpy as np

# CSV 파일에서 데이터 로드: 쉼표로 구분, 실수형으로 불러옴
xy = np.loadtxt('../data-01-test-score.csv', delimiter=',', dtype=np.float32)

# 입력(x): 마지막 열 제외한 나머지 (국어, 영어, 수학 점수)
x_data = xy[:, 0:-1]

# 출력(y): 마지막 열만 (최종 점수)
y_data = xy[:, [-1]]  # 2D 형태 유지를 위해 []를 한 번 더 씀

# 데이터 확인 (모양과 내용 출력)
print(x_data, "\nx_data shape:", x_data.shape)
print(y_data, "\ny_data shape:", y_data.shape)

# Sequential 모델 생성
tf.model = tf.keras.Sequential()

# Dense 레이어 추가: 입력 3개 → 출력 1개, 선형 활성화(linear, 생략 가능하지만 명시적으로 적음)
tf.model.add(tf.keras.layers.Dense(units=1, input_dim=3, activation='linear'))

# 모델 구조 요약 출력
tf.model.summary()

# 모델 컴파일: 평균제곱오차(MSE)를 손실함수로, SGD 옵티마이저 사용 (learning rate는 0.00001)
tf.model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-5))

# 모델 학습: 2000번 반복해서 학습
history = tf.model.fit(x_data, y_data, epochs=2000)

# 새로운 입력 데이터에 대한 예측 수행
print("Your score will be ", tf.model.predict([[100, 70, 101]]))           # 예: 내 점수 예측
print("Other scores will be ", tf.model.predict([[60, 70, 110], [90, 100, 80]]))  # 예: 다른 사람 예측


FileNotFoundError: ../data-01-test-score.csv not found.

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x
